<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Resample function in Vantage
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial'><b>Introduction</b></p>
<p style = 'font-size:16px;font-family:Arial'>The Resample() function transforms an irregular time series into a regular time series. It can also be used to alter the sampling interval for a time series.</p>

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>1. Initiate a connection to Vantage</b>

<p style = 'font-size:16px;font-family:Arial'>In the section, we import the required libraries and set environment variables and environment paths (if required).

In [ ]:
from teradataml import (
    create_context,
    execute_sql,
    load_example_data,
    DataFrame,
    in_schema,
    TDSeries,
    Resample,
    db_drop_view,
    remove_context
    )

# Modify the following to match the specific client environment settings
display.max_rows = 5

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>1.1 Connect to Vantage</b></p>
<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../../UseCases/startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=PP_Resample_Python.ipynb;' UPDATE FOR SESSION; ''')

<p style = 'font-size:16px;font-family:Arial'>Begin running steps with Shift + Enter keys. </p>

<hr style='height:1px;border:none;'>

<p style = 'font-size:18px;font-family:Arial'><b>1.2 Getting Data for This Demo</b></p>

<p style = 'font-size:16px;font-family:Arial'>We have provided data for this demo on cloud storage. You can either run the demo using foreign tables to access the data without any storage on your environment or download the data to local storage, which may yield faster execution. Still, there could be considerations of available storage. Two statements are in the following cell, and one is commented out. You may switch which mode you choose by changing the comment string.</p>

In [ ]:
# %run -i ../../UseCases/run_procedure.py "call get_data('DEMO_SalesForecasting_cloud');"
 # Takes about 45 seconds
%run -i ../../UseCases/run_procedure.py "call get_data('DEMO_SalesForecasting_local');"
 # Takes about 70 seconds

<p style = 'font-size:16px;font-family:Arial'>Next is an optional step – if you want to see the status of databases/tables created and space used.</p>

In [ ]:
%run -i ../../UseCases/run_procedure.py "call space_report();"        # Takes 10 seconds

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>2. Data Exploration</b>
<p style = 'font-size:16px;font-family:Arial'>Create a "Virtual DataFrame" that points to the data set in Vantage. Check the shape of the dataframe as check the datatype of all the columns of the dataframe.</p>

In [ ]:
query2='''REPLACE VIEW Weekly_Sales_Details AS
SELECT
    w.Sales_date AS times,
    CAST('2012-02-03' AS DATE) AS cutoff_date,
    w.Dept,
    w.Store,
    CAST(w.Sales_Date AS TIMESTAMP) AS Sales_Date,
    ZEROIFNULL(Weekly_Sales) AS Weekly_Sales,
    ZEROIFNULL(Store_Size) AS Store_Size,
    Store_Type AS Store_Type,
    w.IsHoliday,
    ZEROIFNULL(Temperature) AS Temperature,
    ZEROIFNULL(MarkDown1) AS MarkDown1,
    ZEROIFNULL(MarkDown2) AS MarkDown2,
    ZEROIFNULL(MarkDown3) AS MarkDown3,
    ZEROIFNULL(MarkDown4) AS MarkDown4,
    ZEROIFNULL(MarkDown5) AS MarkDown5,
    ZEROIFNULL(CPI) AS CPI,
    ZEROIFNULL(Unemployment) AS Unemployment,
    ZEROIFNULL(Fuel_Price) AS Fuel_Price,
    CAST(TRIM(w.Dept) || TRIM(w.Store) AS INT) AS idcols
FROM
    Demo_SalesForecasting.Weekly_Sales w
LEFT JOIN
    Demo_SalesForecasting.Stores s ON w.Store = s.Store
LEFT JOIN
    Demo_SalesForecasting.Features f ON w.Store = f.store AND w.Sales_Date = f.Sales_Date
WHERE
    w.Store IN (20, 4);
'''

execute_sql(query2)
modeldf=DataFrame.from_query('select * from Weekly_Sales_Details;')
modeldf

In [ ]:
modeldf.shape

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>3. Resample</b>
<p style = 'font-size:16px;font-family:Arial'>Execute the Resample function to transform irregular time series into regular time series when row index style is "TIMECODE"</p>

<p style = 'font-size:16px;font-family:Arial'>We need to first convert the data from dataframe into a TDSeries</p>

In [ ]:
from teradataml import Resample
data_series_df = TDSeries(data=modeldf,
                              id="idcols",
                              row_index=("Sales_Date"),
                              row_index_style= "TIMECODE",
                              payload_field="Weekly_Sales",
                              payload_content="REAL")

<p style = 'font-size:16px;font-family:Arial'>We will than apply the Resample function on the TDSeries data</p>

In [ ]:
uaf_out1 = Resample(data=data_series_df,
                        interpolate='LINEAR',
                        timecode_start_value="TIMESTAMP '2010-02-05 00:00:00'",
                        timecode_duration="WEEKS(1)")
resample_df=uaf_out1.result

In [ ]:
resample_df.shape

<p style = 'font-size:16px;font-family:Arial'>As we can see from above, the number of rows after Resample has increased from 20486 to 21400. The Resample function has transformed the irregular time series into a regular time series.</p>

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>4. Cleanup</b>

<p style = 'font-size:18px;font-family:Arial'><b>Work Tables</b></p>
<p style = 'font-size:16px;font-family:Arial'>The following code will clean up intermediate tables.</p>

In [ ]:
db_drop_view('Weekly_Sales_Details')

<p style = 'font-size:18px;font-family:Arial'><b>Databases and Tables</b></p>
<p style = 'font-size:16px;font-family:Arial'>The following code will clean up tables and databases created above.</p>

In [ ]:
%run -i ../../UseCases/run_procedure.py "call remove_data('DEMO_SalesForecasting');" 
#Takes 45 seconds

In [ ]:
remove_context()

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>Dataset</b></p>
<p style = 'font-size:16px;font-family:Arial'>This is the historical data that covers sales from 2010-02-05 to 2012-11-01. Within this file you will find the following fields:</p>

<li style = 'font-size:16px;font-family:Arial'>Store - the store number</li>
<li style = 'font-size:16px;font-family:Arial'>Date - the week of sales</li>
<li style = 'font-size:16px;font-family:Arial'>Weekly_Sales - sales for the given store</li>
<li style = 'font-size:16px;font-family:Arial'>Holiday_Flag - whether the week is a special holiday week 1 – Holiday week 0 – Non-holiday week</li>
<li style = 'font-size:16px;font-family:Arial'>Temperature - Temperature on the day of sale</li>
<li style = 'font-size:16px;font-family:Arial'>Fuel_Price - Cost of fuel in the region</li>
<li style = 'font-size:16px;font-family:Arial'>CPI – Prevailing consumer price index</li>
<li style = 'font-size:16px;font-family:Arial'>Unemployment - Prevailing unemployment rate</li>
<li style = 'font-size:16px;font-family:Arial'>Holiday Events: Super Bowl, Labour Day, Thanksgiving, Christmas </li>
</p>

<p style = 'font-size:16px;font-family:Arial'><b>Links:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li>Teradataml Python reference: <a href = 'https://docs.teradata.com/search/all?query=Python+Package+User+Guide&content-lang=en-US'>here</a></li>
    <li>Resample function reference: <a href = 'https://docs.teradata.com/search/all?query=Resample&content-lang=en-US'>here</a></li>
</ul>

<footer style="padding-bottom:35px; border-bottom:3px solid #91A0Ab">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2025. All Rights Reserved
        </div>
    </div>
</footer>